In [1]:
import psycopg2
import pandas as pd

## Создание БД для выполнения заданий

In [23]:
conn = psycopg2.connect(
            host='127.0.0.1',
            database='sber_test',
            user='postgres',
            password='1234'
        )
cur = conn.cursor()

### Таблица с данными о договорах

In [18]:
cur.execute("""
                CREATE TABLE IF NOT EXISTS loans_table(
                LOAN_ID int,
                CLIENT_ID int,
                LOAN_DATE date,
                LOAN_AMOUNT FLOAT
                );
            """)
cur.execute("""
        INSERT INTO loans_table (LOAN_ID, CLIENT_ID, LOAN_DATE, LOAN_AMOUNT) VALUES
        (1, 1, '2019-01-15', 10000),
        (2, 2, '2020-02-15', 20000),
        (3, 3, '2020-03-15', 30000),
        (4, 4, '2020-04-15', 40000),
        (5, 1, '2020-01-16', 15000),
        (6, 2, '2020-03-15', 35000),
        (7, 3, '2020-03-15', 5000),
        (8, 1, '2020-01-15', 1500),
        (9, 2, '2020-02-15', 500),
        (10, 1, '2020-01-15', 1500);
""")
conn.commit()

### Таблица с данными о клиентах

In [22]:
cur.execute("""
                CREATE TABLE IF NOT EXISTS clients_table(
                CLIENT_ID int,
                CLIENT_NAME varchar(20),
                BIRTHDAY date,
                GENDER VARCHAR(20));
            """)

cur.execute("""
            INSERT INTO clients_table VALUES
            (1,'bob','2020-01-15','male'),
            (2,'rocky','2020-02-15','female'),
            (3,'like','2020-02-15','female'),
            (4,'ricky','2020-02-15','male');
            """)

conn.commit()

### Таблица с контактами

In [24]:
cur.execute("""
                CREATE TABLE IF NOT EXISTS contacts(
                key int,
                id int,
                phone varchar,
                mail varchar
                );
            """)

cur.execute("""
            INSERT INTO contacts VALUES
            (1,12345,89997776655,'test@mail.ru'),
            (2,54321,87778885566,'two@mail.ru'),
            (3,98765,87776664577,'three@mail.ru'),
            (4,66678,87778885566,'four@mail.ru'),
            (5,34567,84547895566,'four@mail.ru'),
            (6,34567,89087545678,'five@mail.ru');
            """)

conn.commit()

## Задание 1.

Вывести для всех строк из таблицы contacts столбец со всеми key соответствующими определенным id в порядке убывания


In [32]:
pd.read_sql("""
WITH keys AS(
    SELECT 
        id, 
        string_agg(KEY::text, ', ') as other_keys
    FROM contacts
    GROUP BY id
    HAVING COUNT(KEY) > 1)
SELECT
	key,
    contacts.id,
    phone,
    mail,
    COALESCE(other_keys, ' ') as other_keys
FROM contacts
LEFT JOIN keys ON keys.id = contacts.id;
""", conn)


C:\Users\фвьшт\AppData\Local\Temp\ipykernel_12184\121452948.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,key,id,phone,mail,other_keys
0,1,12345,89997776655,test@mail.ru,
1,2,54321,87778885566,two@mail.ru,
2,3,98765,87776664577,three@mail.ru,
3,4,66678,87778885566,four@mail.ru,
4,5,34567,84547895566,four@mail.ru,"5, 6"
5,6,34567,89087545678,five@mail.ru,"5, 6"


## Задание 2.

Для каждого клиента вывести информацию: суммарная стоимость его договоров выше среднего его пола или ниже


In [33]:
pd.read_sql("""
WITH gender_amounts AS
  (SELECT 
      gender,
      lt.client_id,
      sum(loan_amount) as sum_amount
  FROM loans_table lt
  JOIN clients_table ct ON ct.client_id = lt.client_id
  GROUP BY gender, lt.client_id)
SELECT
  	client_id,
    gender,
    sum_amount,
    AVG(sum_amount) OVER(PARTITION BY gender) as avg_gender_amount,
    CASE
    	WHEN sum_amount > AVG(sum_amount) OVER(PARTITION BY gender)
        THEN 1
        else 0
    end as more_than_avg_gender
FROM gender_amounts
order by client_id
""", conn)

C:\Users\фвьшт\AppData\Local\Temp\ipykernel_12184\2059796471.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,client_id,gender,sum_amount,avg_gender_amount,more_than_avg_gender
0,1,male,28000.0,34000.0,0
1,2,female,55500.0,45250.0,1
2,3,female,35000.0,45250.0,0
3,4,male,40000.0,34000.0,1


## Задание 3.

Собрать в виде сводной таблицы с разбивкой по полу: сколько первых, вторых и третьих договоров было заключено в 2020 году

In [34]:
pd.read_sql("""
WITH loan_numbers AS (
  SELECT
      lt.client_id,
      ROW_NUMBER() OVER (PARTITION BY lt.client_id ORDER BY loan_date) as loan_number,
      gender
  FROM loans_table lt
  JOIN clients_table ct ON ct.client_id = lt.client_id
  WHERE date_part('year', loan_date::date) = 2020)
  
 SELECT
    gender,
    COUNT(CASE WHEN loan_number = 1 THEN 1 END) as "1",
    COUNT(CASE WHEN loan_number = 2 THEN 1 END) as "2",
    COUNT(CASE WHEN loan_number = 3 THEN 1 END) as "3"
 FROM loan_numbers
 GROUP BY gender
""", conn)

C:\Users\фвьшт\AppData\Local\Temp\ipykernel_12184\1597126910.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,gender,1,2,3
0,female,2,2,1
1,male,2,1,1


## Задание 4.

Для каждого клиента вывести: сумму каждого его договора и сумму всех его договоров накопительным итогом.

Перед каждым новым договором добавьте дополнительную строку с датой, которая предшествует дате договора на 1 день и со значением суммы = NULL, это будет значить, что обязательства договоры исполнены

In [35]:
pd.read_sql("""
WITH loan_row_numbers as (SELECT
     client_id,
     loan_date,
     loan_amount,
     ROW_NUMBER() OVER (PARTITION BY client_id ORDER BY loan_date, loan_id) as rn
FROM loans_table lt),
loan_completed as (SELECT 
	client_id,
    loan_date - interval '1 day' as loan_date,
    NULL::integer as loan_amount,
    rn - 0.5 as rn
FROM loan_row_numbers
WHERE rn != 1),
loans_full_data AS (
SELECT client_id, loan_date, loan_amount, rn FROM loan_row_numbers
UNION all
SELECT * FROM loan_completed)

SELECT 
	client_id,
    loan_date,
    loan_amount,
    SUM(loan_amount) OVER (PARTITION BY client_id ORDER BY rn) as amount_cumsum
FROM loans_full_data
order by client_id, rn
""", conn)

C:\Users\фвьшт\AppData\Local\Temp\ipykernel_12184\2018728802.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,client_id,loan_date,loan_amount,amount_cumsum
0,1,2019-01-15,10000.0,10000.0
1,1,2020-01-14,NaN,10000.0
2,1,2020-01-15,1500.0,11500.0
3,1,2020-01-14,NaN,11500.0
4,1,2020-01-15,1500.0,13000.0
5,1,2020-01-15,NaN,13000.0
6,1,2020-01-16,15000.0,28000.0
7,2,2020-02-15,20000.0,20000.0
8,2,2020-02-14,NaN,20000.0
9,2,2020-02-15,500.0,20500.0


### Задание 5.

Предложите метод аналитики эффективности работы компании, на основании имеющихся данных
* Какие выводы вы могли бы сделать? Приведите несколько вариантов
* Какие дополнительные данные вам нужны, чтобы сделать анализ более эффективным? 

Можно провести:
1. **Когортный анализ в разрезе метрики LTV**: как каждая когорта возвращается и продолжает заключать новые договора? За когорту взять год, так же разделить когорты на сегменты по полу клиента. С помощью данного анализа можно получить информацию о примерном доходе с одного клиента.
2. **RFM анализ:** так мы сможем разделить клиентов по различным сегментам (часто заключают договора на большие суммы, редко заключают договора на большие суммы и т.д.), посмотреть распределение сегментов.

Необходимые дополнительные данные:

1. Прежде всего необходимо понимать предметную область компании, в нескольких заданиях внимание акцентировано на сегментировании по полу клиентов, скорее всего есть значимые различия в заключении договоров между мужчинами и женщинами.

2. Из понимания предметной области также можно внести корректировки в когортный анализ, анализ Retention, в контексте периода возвращения, возможно для компании нормальна ситуация, когда клиенты заключают договоры не раз в полгода, а раз в год или два, и отслеживать нужно эти периоды.

3. Также необходима информация о стоимости привлечения клиентов (CAC), чтобы сравнить ее с LTV и сделать выводы о рентабельности продукта.

Пример когортного анализа метрики LTV на имеющихся данных:

In [36]:
pd.read_sql("""
WITH cohorts AS(
SELECT 
	lt.client_id,
    gender,
    loan_amount,
    FIRST_VALUE(date_part('year', loan_date)) OVER (PARTITION BY lt.client_id ORDER BY loan_date) as cohort,
    loan_date - FIRST_VALUE(loan_date) OVER (PARTITION BY lt.client_id ORDER BY loan_date) as diff
FROM loans_table lt
LEFT JOIN clients_table ct ON ct.client_id = lt.client_id)

SELECT
	cohort,
    gender,
    SUM(CASE WHEN diff >= 0 and diff < 180 THEN loan_amount END) AS "0-180",
    SUM(CASE WHEN diff >= 180 and diff < 365 THEN loan_amount END) AS "180-365",
    SUM(CASE WHEN diff >= 365 and diff < 500 THEN loan_amount END) AS "365-500"
FROM cohorts
GROUP BY cohort, gender 
ORDER BY cohort
""", conn)


C:\Users\фвьшт\AppData\Local\Temp\ipykernel_12184\668157497.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,cohort,gender,0-180,180-365,365-500
0,2019.0,male,10000.0,None,18000.0
1,2020.0,female,90500.0,None,NaN
2,2020.0,male,40000.0,None,NaN


Женщины склонны заключать договора на более крупные суммы, однако не возвращаются спустя время.

В то же время мужчины заключают договора на меньшие суммы и охотнее заключают новые договора спустя время. 

> Когорт и данных мало, поэтому выводы нерепрезентативны. Это пример рассуждений после проведения когортного анализа LTV.